In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.13-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive # 此时colab中出现drive的文件夹，里面就是你的google drive的根目录文件

In [0]:
!ls -al

total 32
drwxr-xr-x 1 root root 4096 Oct 17 15:18 .
drwxr-xr-x 1 root root 4096 Oct 17 15:16 ..
-rw-r--r-- 1 root root 2661 Oct 17 15:18 adc.json
drwxr-xr-x 1 root root 4096 Oct 17 15:18 .config
drwxr-xr-x 2 root root 4096 Oct 17 15:18 drive
drwxr-xr-x 1 root root 4096 Aug 27 16:17 sample_data


In [0]:
import os
os.chdir("drive/Colab Notebooks/Team management/ccf2016_sougou")

In [0]:
!ls -al

total 89
drwxr-xr-x 2 root root  4096 Oct 15 08:25  .
drwxr-xr-x 2 root root  4096 Oct 15 01:58  ..
-rw-r--r-- 1 root root  6349 Oct 17 14:52 '1.NLP实战项目 -- 基于自然语言处理的搜索引擎公司用户画像.ipynb'
-rw-r--r-- 1 root root 37842 Oct 17 14:52  2.项目思路.ipynb
-rw-r--r-- 1 root root  2993 Oct 17 14:52  d2v_model.py
drwxr-xr-x 2 root root  4096 Oct 17 14:52  data
-rw-r--r-- 1 root root  2537 Oct 17 14:52  dbow_nn_stack.py
-rw-r--r-- 1 root root  2363 Oct 17 14:52  dm_nn_stack.py
-rw-r--r-- 1 root root  2328 Oct 17 14:52  fill_nan.py
drwxr-xr-x 2 root root  4096 Oct 17 14:52  .idea
drwxr-xr-x 2 root root  4096 Oct 17 14:52  img
drwxr-xr-x 2 root root  4096 Oct 17 14:52  .ipynb_checkpoints
-rw-r--r-- 1 root root  3972 Oct 17 15:18  run_sh.ipynb
-rw-r--r-- 1 root root  2859 Oct 17 14:52  tfidf_lr_stack.py
-rw-r--r-- 1 root root  3563 Oct 17 14:52  xgb_ens.py


In [0]:
# 作者将三个属性中为0（代表未知）的数据进行处理，通过不为0的数据使用logistic regression进行预测后填充
# query的特征表示使用TF-IDF
# 处理完的文件为all_v2.csv
import pandas as pd
import numpy as np
import jieba
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

#filter warning
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# 读取原始数据，只取前1w条
df_tr = pd.read_csv('./data/train.csv', sep="###__###", header=None, nrows=10000)
df_tr.columns = ['ID', 'Age', 'Gender', 'Education', 'query']
df_te = pd.read_csv('./data/test.csv', sep="###__###", header=None, nrows=10000)
df_te.columns = ['ID', 'query']
print(df_tr.shape)
print(df_te.shape)

df_all = df_tr
for lb in ['Education', 'Age', 'Gender']:
    df_all[lb] = df_all[lb] - 1
    print(df_all.iloc[:10000][lb].value_counts())


class Tokenizer:
    def __init__(self):
        self.n = 0

    def __call__(self, line, *args, **kwargs):
        tokens = []
        for query in line.split('\t'):
            words = [word for word in jieba.cut(query)]
            for gram in [1, 2]:
                for i in range(len(words) - gram + 1):
                    tokens += ['_*_'.join(words[i:i+gram])]
        if np.random.rand() < 0.00001:
            print(line)
            print('='*20)
            print(tokens)
        self.n += 1
        if self.n % 1000 == 0:
            print(self.n, end=' ')
        return tokens


tfv = TfidfVectorizer(tokenizer=Tokenizer(), min_df=3, max_df=0.95, sublinear_tf=True)
X_sp = tfv.fit_transform(df_all['query'])
print(len(tfv.vocabulary_))
X_all = X_sp

lb = 'Education'
idx = 3
tr = np.where(df_all[lb] != -1)[0]
va = np.where(df_all[lb] == -1)[0]
df_all.iloc[va, idx] = \
    LogisticRegression(C=1).fit(X_all[tr], df_all.iloc[tr, idx]).predict(X_all[va])

lb = 'Age'
idx = 1
tr = np.where(df_all[lb] != -1)[0]
va = np.where(df_all[lb] == -1)[0]
df_all.iloc[va, idx] = \
    LogisticRegression(C=2).fit(X_all[tr], df_all.iloc[tr, idx]).predict(X_all[va])

lb = 'Gender'
idx = 2
tr = np.where(df_all[lb]!=-1)[0]
va = np.where(df_all[lb]==-1)[0]
df_all.iloc[va,idx] = \
    LogisticRegression(C=2).fit(X_all[tr], df_all.iloc[tr, idx]).predict(X_all[va])

df_all = pd.concat([df_all, df_te]).fillna(0)
df_all.to_csv('./data/all_v2.csv', index=None, encoding='utf8')



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


(10000, 5)
(10000, 2)
 4    3764
 3    2804
 2    1869
-1     929
 5     540
 1      55
 0      39
Name: Education, dtype: int64
 0    3920
 1    2693
 2    1819
 3    1036
 4     314
-1     173
 5      45
Name: Age, dtype: int64
 0    5720
 1    4048
-1     232
Name: Gender, dtype: int64


Loading model cost 0.762 seconds.
Prefix dict has been built succesfully.


1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 240899


In [0]:
# 训练doc2vec模型，dm和dbow都进行了训练
import pandas as pd
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# ********************************************************************************
# =========================================== #
# 对query进行分词, 保存为query_cut.csv文件。     #
# 该过程较为耗时，无需反复执行，运行一次后即可注释掉 #
# 再次使用直接读取                              #
# =========================================== #
# df_query = pd.read_csv('./data/all_v2.csv', usecols=['query'], encoding='utf-8')
# rows = []
# for i, line in enumerate(df_query.iloc[:10000]['query']):
#     words = []
#     row = {}
#     for query in line.split('\t'):
#         words.extend(list(jieba.cut(query)))
#     row['tag'] = i
#     row['query_cut'] = words
#     rows.append(row)
# df_query_cut = pd.DataFrame(rows)
# df_query_cut.to_csv('./data/query_cut.csv', index=None, encoding='utf8')
# ***********************************************************************************

df_query_cut = pd.read_csv('./data/query_cut.csv', encoding='utf-8')
query_tagged = df_query_cut.apply(
    lambda line: TaggedDocument(words=line['query_cut'], tags=[line['tag']]), axis=1)

df_lb = pd.read_csv('./data/all_v2.csv',
                    usecols=['Education', 'Age', 'Gender'], nrows=10000)
ys = {}
for lb in ['Education', 'Age', 'Gender']:
    ys[lb] = np.array(df_lb[lb])

# -------------------train dbow doc2vec---------------------------------------------
dbow_model = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=3,
                     window=30, sample=1e-5, workers=8, alpha=0.025, min_alpha=0.025)
dbow_model.build_vocab(query_tagged)

for i in range(2):
    print('pass:', i + 1)
    dbow_model.train(query_tagged, total_examples=dbow_model.corpus_count, epochs=dbow_model.epochs)
    X_d2v = np.array([dbow_model.docvecs[i] for i in range(10000)])
    for lb in ['Education', 'Age', 'Gender']:
        scores = cross_val_score(LogisticRegression(C=3), X_d2v, ys[lb], cv=5)
        print('dbow', lb, scores, np.mean(scores))
dbow_model.save('./data/dbow_model.model')

print('='*20)

# -------------------train dm doc2vec---------------------------------------------
dm_model = Doc2Vec(dm=1, vector_size=300, negative=5, hs=0, min_count=3,
                   window=10, sample=1e-5, workers=8, alpha=0.025, min_alpha=0.025)
dm_model.build_vocab(query_tagged)

for i in range(2):
    print('pass:', i + 1)
    dm_model.train(query_tagged, total_examples=dm_model.corpus_count, epochs=dm_model.epochs)
    X_d2v = np.array([dm_model.docvecs[i] for i in range(10000)])
    for lb in ['Education', 'Age', 'Gender']:
        scores = cross_val_score(LogisticRegression(C=3), X_d2v, ys[lb], cv=5)
        print('dm', lb, scores, np.mean(scores))
dm_model.save('./data/dm_model.model')

pass: 1
dbow Education [0.56821589 0.58170915 0.556      0.564      0.56706707] 0.5673984209096654
dbow Age [0.51998002 0.51574213 0.51524238 0.51525763 0.5092639 ] 0.515097210476864
dbow Gender [0.794  0.791  0.782  0.8005 0.7835] 0.7902
pass: 2
dbow Education [0.56571714 0.56021989 0.559      0.5545     0.57607608] 0.5631026215120668
dbow Age [0.51848152 0.51424288 0.50524738 0.52576288 0.52228343] 0.5172036159865019
dbow Gender [0.799  0.799  0.7885 0.8095 0.7875] 0.7967


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


pass: 1
dm Education [0.55722139 0.55922039 0.541      0.5595     0.55055055] 0.5534984659321991
dm Age [0.500999   0.49375312 0.50974513 0.50275138 0.48923385] 0.49929649566751444
dm Gender [0.778  0.779  0.762  0.778  0.7725] 0.7739
pass: 2
dm Education [0.55322339 0.56121939 0.5575     0.56       0.55605606] 0.5575997669333501
dm Age [0.50849151 0.49175412 0.50924538 0.49974987 0.51677516] 0.5052032092845937
dm Gender [0.781 0.78  0.78  0.786 0.777] 0.7808


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
import pandas as pd
import numpy as np
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold


# -------------定义评估函数---------------
def myAcc(y_true, y_pred):
    y_pred = np.argmax(y_pred, axis=1)
    return np.mean(y_true == y_pred)


# load data
df_all = pd.read_csv('./data/all_v2.csv', encoding='utf-8', nrows=10000)
ys = {}
for lb in ['Education', 'Age', 'Gender']:
    ys[lb] = np.array(df_all[lb])

# ********************************************************************************
# ================================================ #
# 对query使用TF-IDF进行表示，保存为tfidf_1w.feat文件.  #
# 该过程较为耗时，无需反复执行，运行一次后即可注释掉      #
# 再次使用直接用pickle读取                            #
# ================================================ #
# class Tokenizer:
#     def __init__(self):
#         self.n = 0
#
#     def __call__(self, line, *args, **kwargs):
#         tokens = []
#         for query in line.split('\t'):
#             words = [word for word in jieba.cut(query)]
#             for gram in [1, 2]:
#                 for i in range(len(words) - gram + 1):
#                     tokens += ['_*_'.join(words[i:i+gram])]
#         self.n += 1
#         if self.n % 1000 == 0:
#             print(self.n)
#         return tokens
#
#
# tfv = TfidfVectorizer(tokenizer=Tokenizer(), min_df=3, max_df=0.95, sublinear_tf=True)
# X_sp = tfv.fit_transform(df_all['query'])
# pickle.dump(X_sp, open('./data/tfidf_1w.pkl', 'wb'))
# ********************************************************************************

X_sp = pickle.load(open('./data/tfidf_1w.feat', 'rb'))
df_stack = pd.DataFrame(index=range(len(df_all)))

# -----------------------stack for education/age/gender------------------
for lb in ['Education', 'Age', 'Gender']:
    print(lb)
    TR = 8000
    num_class = len(pd.value_counts(ys[lb]))
    n = 5

    X = X_sp[:TR]
    y = ys[lb][:TR]
    X_te = X_sp[TR:]
    y_te = ys[lb][TR:]

    stack = np.zeros((X.shape[0], num_class))
    stack_te = np.zeros((X_te.shape[0], num_class))

    kfold = KFold(n_splits=n, random_state=2019)
    for i, (tr, va) in enumerate(kfold.split(X)):
        print('stack:%d/%d' % (i + 1, n))
        clf = LogisticRegression(C=3)
        clf.fit(X[tr], y[tr])
        y_pred_va = clf.predict_proba(X[va])
        y_pred_te = clf.predict_proba(X_te)
        print('va acc:', myAcc(y[va], y_pred_va))
        print('te acc:', myAcc(y_te, y_pred_te))
        stack[va] += y_pred_va
        stack_te += y_pred_te
    stack_te /= n
    stack_all = np.vstack([stack, stack_te])
    for i in range(stack_all.shape[1]):
        df_stack['tfidf_{}_{}'.format(lb, i)] = stack_all[:, i]

df_stack.to_csv('./data/tfidf_stack_1w.csv', index=None, encoding='utf-8')
print('done!')

Education
stack:1/5
va acc: 0.605
te acc: 0.635
stack:2/5
va acc: 0.64625
te acc: 0.6295
stack:3/5
va acc: 0.6175
te acc: 0.6355
stack:4/5
va acc: 0.625625
te acc: 0.6275
stack:5/5
va acc: 0.6125
te acc: 0.632
Age
stack:1/5
va acc: 0.535
te acc: 0.572
stack:2/5
va acc: 0.56625
te acc: 0.5645
stack:3/5
va acc: 0.56
te acc: 0.5575
stack:4/5
va acc: 0.545
te acc: 0.5705
stack:5/5
va acc: 0.558125
te acc: 0.5705
Gender
stack:1/5
va acc: 0.821875
te acc: 0.8045
stack:2/5
va acc: 0.824375
te acc: 0.8075
stack:3/5
va acc: 0.79875
te acc: 0.809
stack:4/5
va acc: 0.823125
te acc: 0.809
stack:5/5
va acc: 0.81625
te acc: 0.8135
done!


In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from gensim.models import Doc2Vec

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.utils import to_categorical


def myAcc(y_true, y_pred):
    y_pred = np.argmax(y_pred, axis=1)
    return np.mean(y_true == y_pred)


df_all = pd.read_csv('./data/all_v2.csv', usecols=['ID', 'Education', 'Age', 'Gender'], nrows=10000)
ys = {}
for label in ['Education', 'Age', 'Gender']:
    ys[label] = np.array(df_all[label])

model = Doc2Vec.load('./data/dbow_model.model')
X_sp = np.array([model.docvecs[i] for i in range(len(df_all))])

df_stack = pd.DataFrame(index=range(len(df_all)))
TR = 8000
n = 5
X = X_sp[:TR]
X_te = X_sp[TR:]

for i, lb in enumerate(['Education', 'Age', 'Gender']):
    print(lb)
    num_class = len(pd.value_counts(ys[lb]))
    y = ys[lb][:TR]
    y_te = ys[lb][TR:]

    stack = np.zeros((X.shape[0], num_class))
    stack_te = np.zeros((X_te.shape[0], num_class))

    nn_model = Sequential()
    nn_model.add(Dense(300, input_shape=(X.shape[1],)))
    nn_model.add(Dropout(0.1))
    nn_model.add(Activation('tanh'))
    nn_model.add(Dense(num_class))
    nn_model.add(Activation('softmax'))

    nn_model.compile(
        optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

    kfold = KFold(n_splits=n, random_state=2019)
    for i, (tr, va) in enumerate(kfold.split(X)):
        print('stack:{}/{}'.format(i + 1, n))
        nb_classes = num_class
        X_train = X[tr].astype('float32')
        y_train = y[tr]
        Y_train = to_categorical(y_train, num_classes=nb_classes)
        X_test = X_te.astype('float32')
        y_test = y_te
        Y_test = to_categorical(y_test, num_classes=nb_classes)

        history = nn_model.fit(X_train, Y_train,
                               shuffle=True, batch_size=128, epochs=16, verbose=2,
                               validation_data=(X_test, Y_test))

        y_pred_va = nn_model.predict_proba(X[va])
        y_pred_te = nn_model.predict_proba(X_te)
        print('va acc: ', myAcc(y[va], y_pred_va))
        print('te acc: ', myAcc(y_te, y_pred_te))
        stack[va] += y_pred_va
        stack_te += y_pred_te
    stack_te /= n
    stack_all = np.vstack([stack, stack_te])
    for i in range(stack_all.shape[1]):
        df_stack['dbow_nn_{}_{}'.format(lb, i)] = stack_all[:, i]

df_stack.to_csv('./data/dbow_stack_1w.csv', index=None, encoding='utf-8')
print('done!')



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Education
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
stack:1/5
Train on 6400 samples, validate on 2000 samples
Epoch 1/16
6400/6400 - 1s - loss: 1.8186 - acc: 0.1883 - val_loss: 1.8186 - val_acc: 0.1835
Epoch 2/16
6400/6400 - 0s - loss: 1.8069 - acc: 0.1925 - val_loss: 1.8085 - val_acc: 0.1875
Epoch 3/16
6400/6400 - 0s - loss: 1.7983 - acc: 0.1992 - val_loss: 1.7984 - val_acc: 0.1970
Epoch 4/16
6400/6400 - 0s - loss: 1.7883 - acc: 0.2158 - val_loss: 1.7883 - val_acc: 0.2055
Epoch 5/16
6400/6400 - 0s - loss: 1.7778 - acc: 0.2183 - val_loss: 1.7781 - val_acc: 0.2110
Epoch 6/16
6400/6400 - 0s - loss: 1.7674 - acc: 0.2277 - val_loss: 1.7679 - val_acc: 0.2220
Epoch 7/16
6400/6400 - 0s - loss: 1.7618 - acc: 0.2345 - val_loss: 1.7577 - val_acc: 0.2330
Epoch 8/16
6400/6400 - 0s - loss: 1.7511 - acc: 0.2458 - val_loss: 1.7476 - val_acc: 0.2460
Epoch 9/16
6400/6400 - 0s - loss: 1.7402 - acc: 0.2489 - val_loss: 1.7375 - val_acc: 0.2515
Epoch 10/16
6400/6400 -

In [0]:
import pandas as pd
import numpy as np
from gensim.models import Doc2Vec
from sklearn.model_selection import KFold

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.utils import to_categorical


def myAcc(y_true, y_pred):
    y_pred = np.argmax(y_pred, axis=1)
    return np.mean(y_true == y_pred)


df_all = pd.read_csv(
    './data/all_v2.csv', usecols=['ID', 'Education', 'Age', 'Gender'], nrows=10000)
ys = {}
for lb in ['Education', 'Age', 'Gender']:
    ys[lb] = np.array(df_all[lb])

model = Doc2Vec.load('./data/dm_model.model')
X_sp = np.array([model.docvecs[i] for i in range(len(df_all))])

df_stack = pd.DataFrame(index=range(len(df_all)))
TR = 8000
n = 5

X = X_sp[:TR]
X_te = X_sp[TR:]

for i, lb in enumerate(['Education', 'Age', 'Gender']):
    print(lb)
    num_class = len(pd.value_counts(ys[lb]))
    y = ys[lb][:TR]
    y_te = ys[lb][TR:]

    stack = np.zeros((X.shape[0], num_class))
    stack_te = np.zeros((X_te.shape[0], num_class))

    nn_model = Sequential()
    nn_model.add(Dense(300, input_shape=(X.shape[1], )))
    nn_model.add(Dropout(0.1))
    nn_model.add(Activation('tanh'))
    nn_model.add(Dense(num_class))
    nn_model.add(Activation('softmax'))

    nn_model.compile(
        optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

    kfold = KFold(n_splits=n, random_state=2019)
    for i, (tr, va) in enumerate(kfold.split(X)):
        print('stack:{}/{}'.format(i + 1, n))
        X_train = X[tr]
        Y_train = to_categorical(y[tr], num_class)
        X_test = X_te
        Y_test = to_categorical(y_te, num_class)

        history = nn_model.fit(X_train, Y_train,
                               batch_size=128, epochs=16, shuffle=True, verbose=2,
                               validation_data=(X_test, Y_test))
        y_pred_va = nn_model.predict_proba(X[va])
        y_pred_te = nn_model.predict_proba(X_te)
        print('va acc:', myAcc(y[va], y_pred_va))
        print('te acc:', myAcc(y_te, y_pred_te))

        stack[va] += y_pred_va
        stack_te += y_pred_te
    stack_te /= n
    stack_all = np.vstack([stack, stack_te])
    for i in range(stack_all.shape[1]):
        df_stack['dm_nn_{}_{}'.format(lb, i)] = stack_all[:, i]
df_stack.to_csv('./data/dm_stack_1w.csv', index=None)
print('done!')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Education
stack:1/5
Train on 6400 samples, validate on 2000 samples
Epoch 1/16
6400/6400 - 0s - loss: 2.0044 - acc: 0.1242 - val_loss: 1.9761 - val_acc: 0.1300
Epoch 2/16
6400/6400 - 0s - loss: 1.9855 - acc: 0.1359 - val_loss: 1.9566 - val_acc: 0.1405
Epoch 3/16
6400/6400 - 0s - loss: 1.9651 - acc: 0.1445 - val_loss: 1.9372 - val_acc: 0.1490
Epoch 4/16
6400/6400 - 0s - loss: 1.9466 - acc: 0.1506 - val_loss: 1.9180 - val_acc: 0.1545
Epoch 5/16
6400/6400 - 0s - loss: 1.9283 - acc: 0.1584 - val_loss: 1.8988 - val_acc: 0.1625
Epoch 6/16
6400/6400 - 0s - loss: 1.9104 - acc: 0.1683 - val_loss: 1.8798 - val_acc: 0.1745
Epoch 7/16
6400/6400 - 0s - loss: 1.8887 - acc: 0.1811 - val_loss: 1.8611 - val_acc: 0.1825
Epoch 8/16
6400/6400 - 0s - loss: 1.8741 - acc: 0.1875 - val_loss: 1.8426 - val_acc: 0.1905
Epoch 9/16
6400/6400 - 0s - loss: 1.8531 - acc: 0.2002 - val_loss: 1.8243 - val_acc: 0.2030
Epoch 10/16
6400/6400 - 0s - loss: 1.8439 - acc: 0.2033 - val_loss: 1.8063 - val_acc: 0.2145
Epoch 11/16

In [0]:
import pandas as pd
import numpy as np
import xgboost as xgb


def xgb_acc_score(preds, dtrain):
    y_true = dtrain.get_label()
    y_pred = np.argmax(preds, axis=1)
    return [('acc', np.mean(y_true == y_pred))]


df_lr = pd.read_csv('./data/tfidf_stack_1w.csv')
df_dm = pd.read_csv('./data/dm_stack_1w.csv')
df_dbow = pd.read_csv('./data/dbow_stack_1w.csv')

df_lb = pd.read_csv('./data/all_v2.csv', usecols=['ID', 'Education', 'Age', 'Gender'], nrows=10000)
ys = {}
for lb in ['Education', 'Age', 'Gender']:
    ys[lb] = np.array(df_lb[lb])

df = pd.concat([df_lr, df_dm, df_dbow], axis=1)
print(df.columns)

TR = 8000
df_sub = pd.DataFrame()
df_sub['ID'] = df_lb.iloc[TR:]['ID']
seed = 10

# -------------------------education----------------------------------
lb = 'Education'
print(lb)

esr = 25
evals = 1
n_trees = 1000

num_class = len(pd.value_counts(ys[lb]))
X = df.iloc[:TR]
y = ys[lb][:TR]
X_te = df.iloc[TR:]
y_te = ys[lb][TR:]

ss = 0.9
mc = 2
md = 8
gm = 2

params = {
    'objective': 'multi:softprob',
    'booster': 'gbtree',
    'num_class': num_class,
    'max_depth': md,
    'min_child_weight': mc,
    'subsample': ss,
    'colsample_bytree': 0.8,
    'gamma': gm,
    'eta': 0.01,
    'lambda': 0,
    'alpha': 0,
    'silent': 1
}

dtrain = xgb.DMatrix(X, y)
dvalid = xgb.DMatrix(X_te, y_te)
watchlist = [(dtrain, 'dtrain'), (dvalid, 'eval')]
bst = xgb.train(params, dtrain, n_trees, evals=watchlist, feval=xgb_acc_score, maximize=True,
                early_stopping_rounds=esr, verbose_eval=evals)
df_sub['Education'] = np.argmax(bst.predict(dvalid), axis=1) + 1

# ------------------------ age-----------------------------------
lb = 'Age'
print(lb)
num_class = len(pd.value_counts(ys[lb]))

num_class = len(pd.value_counts(ys[lb]))
X = df.iloc[:TR]
y = ys[lb][:TR]
X_te = df.iloc[TR:]
y_te = ys[lb][TR:]

ss = 0.5
mc = 3
md = 7
gm = 2

params = {
    "objective": "multi:softprob",
    "booster": "gbtree",
    "num_class": num_class,
    'max_depth': md,
    'min_child_weight': mc,
    'subsample': ss,
    'colsample_bytree': 1,
    'gamma': gm,
    "eta": 0.01,
    "lambda": 0,
    'alpha': 0,
    "silent": 1,
}

dtrain = xgb.DMatrix(X, y)
dvalid = xgb.DMatrix(X_te, y_te)
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
bst = xgb.train(params, dtrain, n_trees, evals=watchlist, feval=xgb_acc_score, maximize=True,
                early_stopping_rounds=esr, verbose_eval=evals)
df_sub['Age'] = np.argmax(bst.predict(dvalid), axis=1)+1

# --------------------------gender-------------------------------------
lb = 'Gender'
print(lb)
num_class = len(pd.value_counts(ys[lb]))

num_class = len(pd.value_counts(ys[lb]))
X = df.iloc[:TR]
y = ys[lb][:TR]
X_te = df.iloc[TR:]
y_te = ys[lb][TR:]

ss = 0.5
mc = 0.8
md = 7
gm = 1

params = {
    "objective": "multi:softprob",
    "booster": "gbtree",
    "num_class": num_class,
    'max_depth': md,
    'min_child_weight': mc,
    'subsample': ss,
    'colsample_bytree': 1,
    'gamma': gm,
    "eta": 0.01,
    "lambda": 0,
    'alpha': 0,
    "silent": 1,
}

dtrain = xgb.DMatrix(X, y)
dvalid = xgb.DMatrix(X_te, y_te)
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
bst = xgb.train(params, dtrain, n_trees, evals=watchlist, feval=xgb_acc_score, maximize=True,
                early_stopping_rounds=esr, verbose_eval=evals)
df_sub['Gender'] = np.argmax(bst.predict(dvalid), axis=1) + 1

# df_sub = df_sub[['ID','Age','Gender','Education']]  # 不知道为什么要加这一句，这样就将三个属性直接打乱了
df_sub.to_csv('./data/tfidf_dm_dbow_2k.csv', index=None)



Index(['tfidf_Education_0', 'tfidf_Education_1', 'tfidf_Education_2',
       'tfidf_Education_3', 'tfidf_Education_4', 'tfidf_Education_5',
       'tfidf_Age_0', 'tfidf_Age_1', 'tfidf_Age_2', 'tfidf_Age_3',
       'tfidf_Age_4', 'tfidf_Age_5', 'tfidf_Gender_0', 'tfidf_Gender_1',
       'dm_nn_Education_0', 'dm_nn_Education_1', 'dm_nn_Education_2',
       'dm_nn_Education_3', 'dm_nn_Education_4', 'dm_nn_Education_5',
       'dm_nn_Age_0', 'dm_nn_Age_1', 'dm_nn_Age_2', 'dm_nn_Age_3',
       'dm_nn_Age_4', 'dm_nn_Age_5', 'dm_nn_Gender_0', 'dm_nn_Gender_1',
       'dbow_nn_Education_0', 'dbow_nn_Education_1', 'dbow_nn_Education_2',
       'dbow_nn_Education_3', 'dbow_nn_Education_4', 'dbow_nn_Education_5',
       'dbow_nn_Age_0', 'dbow_nn_Age_1', 'dbow_nn_Age_2', 'dbow_nn_Age_3',
       'dbow_nn_Age_4', 'dbow_nn_Age_5', 'dbow_nn_Gender_0',
       'dbow_nn_Gender_1'],
      dtype='object')
Education
[0]	dtrain-merror:0.294	eval-merror:0.3905	dtrain-acc:0.706	eval-acc:0.6095
Multiple eval me